# Bg2Vec

## Intro

Bg2Vec is a personal project which attempts to apply the [llm2vec](https://arxiv.org/abs/2404.05961) technique on the [BgGPT](https://huggingface.co/INSAIT-Institute/BgGPT-7B-Instruct-v0.2) model.

This notebook will attempt to explain all relevant aspects from scratch, so that a newcomer can understand what the project is all about and gain a familiarity with Large Language Models (LLMs).

## Why me?

[CV](https://drive.google.com/file/d/1T8Vv4-aRrnL12_2ikcN5gPQ-FK7HSLHZ/view?usp=sharing)

## What is a Language Model?

A language model is а function, which can take a piece of text (aka a prompt) and predict the next token. 
This is also known as the Next Token Prediction problem.

We can have a character-based language model, which aims to predict the next character:

E.g. I like to move it, mo ======> should produce something like 

| char | probability |
| ----- | -----
| v | 80%|
| r | 10% |
| o | 5% |
| ... | ...| 

Then we can autoregressively sample from this probability distribution to generate text.

I like to move it, mov ====> e     
I like to move it, move ====> ' '    
I like to move it, move ====> i    

and so on...

We can also work on the word level. Most modern language models work on the subword level, where words are broken into subwords. The process is similar to a breakdown into stems, prefixes and suffixes, but it is statistically derived. 

Let's try this with BgGPT!
 

## Problems with a Language Model

The main problems with a language model are:

1. Hallucinations:
```
<s> [INST] Какъв е текста на химна на България?[/INST]Текстът на химна на България е:

Горда Стара планина,
До нейния връх на свобода,
Нека знамето се вее,
На билото му гордо да се вее.

През годините на робство,
То беше нашата надежда.
Под него се биеха нашите бащи,
За свобода и човешка правда.

Съберете се, всички братя,
Да пеем тази

```

2. Problems integrating new knowledge

E.g. who is the prime minister of Bulgaria?


A major improvement: Retrieval Augmented generation.

## Retrieval Augmented Generation

RAG is a gaining more and more traction nowadays. The basic idea is as follows:

1. Retrieve a set of relevant documents
2. Feed the relevant documents to the LLM and instruct it to use the information to answer your question.

Essentially, instead of asking 
> Who is the prime minister of Bulgaria?

we can ask the following:

> Прочети документите и отговори на въпроса след тях.    
> Документ 1: Димитър Главчев е избран от президента Радев за служебен министър-председател...
> Документ 2: Със 132 гласа Николай Денков беше избран за премиер, за кабинета - 131    
> Въпрос: Кой е министър председател на България?

So, if we have the relevant documents, it is easy. The problem is in the retrieval step. We need a way to index and query documents, so that we can find the relevant information.

A keyword based search is a pretty good option. But we are here to discuss a more recent approach: using the activations from the LLM. 


## How can we use a LLM to encode documents? LLM2Vec!

LLMs process the text in an attempt to predict the next token. By processing thousands of documents, the models have "learned" how to extract the semantics of the text and encode it in the layers of the neural net. We can extract these latent representations, also known as activations and use them to encode the text. This way, similar sentences will be mapped to similar representations.

We can think of a few simple rules to extract a vector which could serve as a representative for the entire document:

1. Use the activations from the last token
2. Aggregate the activations from all tokens, e.g. mean, max, min, etc.
3. Take some kind of weighted sum of the activations

While these approaches are viable, they can certainly be improved. The problem is that the ML models are really good at what they were trained to do one thing and do it very well. In this case, it is to generate text in a sequential manner. An important aspect is that the models are trained with something called **causal attention**, which only allows a given token to see tokens that are before it.

On the one hand, if we use the last token, we have no guarantee that the activations will actually be representative of the document. Their function is simply to predict the next token.

On the other hand, if we use some kind of aggregation, then most of the activations haven't even had the chance to attend to the entire document, so their contribution will not be truly representative as well.

The authors of the LLM2Vec paper recognize these problems and propose the following procedure, which can improve the text encoding capabilities of any recommender:

1. Allow bidirectional attention
2. Finetune via MNTP
3. Finetune via SimCSE

Let's dive deeper into this techniques.

## From Next Token Prediction to Masked Next Token Prediction

![NTP](images/ntp.png) 
<hr>

![MLM](images/mlm.png)

<hr> 


![MNTP](images/mntp.png)

## CLIP & SimCSE

[Contrastive Language–Image Pre-training (CLIP)](https://openai.com/index/clip/) - a model which aims to do produce a pair of encoders which can encode images and text in the same latent space.

It is trained on thousands of <image, description> pairs scraped from the internet.

`<img src="jedihobbit.png" alt ="A Jedi hobbit" >`

<img src="images/jedi-hobbit.jpeg" alt="A Jedi Hobbit" width='500px'>


The model is trained to produce encodings so that the image embeddings are close to their corresponding captions, but far from the captions of other images.

See FastAI Lesson 9: Deep Learning Foundations to Stable Diffusion https://youtu.be/_7rMfsA24Ls?t=6865


<hr> 

[SimCSE: Simple Contrastive Learning of Sentence Embeddings](https://arxiv.org/abs/2104.08821) leverages the same procedure to finetune a LLM to produce better text encodings.


The only problem is: where do we get the pairs? The proposed solution: producing two representations from the same sentence but with different dropout masks.

![SimCSE](images/simcse.png)

 

## Results

![results](images/llm2vec-results.png)

## Progress

My progress so far:

1. I have downloaded and cleaned up the Bulgarian Wikipedia. You can access it here: https://huggingface.co/datasets/mboyanov/bgwiki
2. I have finetuned a Bi-MNTP model starting from the BgGPT model.

Next steps:

1. Train a SimCSE model.
2. Evaluate the resulting models.

## Finale

Време е за ....

| word | probability |
| ----- | -----
| въпроси | 20%|
| бира | 40% |
| манджа | 40% |
| ... | ...| 

# 